## Investigating Gate Teleportation Using Non-Maximally Entangled (NME) States for Gate Cutting

In [1]:
from sympy import *
from sympy.physics.quantum import gate
from sympy.physics.quantum.tensorproduct import TensorProduct
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from IPython.display import Math

# Define symbols
k = symbols('k', real=True)
a,b,c,d = symbols('a b c d', complex=True)
u0, u1, u2, u3 = symbols('u0 u1 u2 u3', complex=True)

# Define gate matricies
H = gate.HadamardGate().get_target_matrix()
X = gate.XGate().get_target_matrix()
Y = gate.YGate().get_target_matrix()
Z = gate.ZGate().get_target_matrix()
SGATE = gate.S().get_target_matrix()
T = gate.T().get_target_matrix()
U = Matrix([[u0, u1], [u2, u3]])

def controlled_gate(gate_matrix):
    """Create a controlled gate matrix from a single qubit gate matrix."""
    return Matrix([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, gate_matrix[0, 0], gate_matrix[0, 1]],
        [0, 0, gate_matrix[1, 0], gate_matrix[1, 1]]
    ])

# Define controlled gate matricies
CH = controlled_gate(H)
CX = controlled_gate(X)
CY = controlled_gate(Y)
CZ = controlled_gate(Z)
CU = controlled_gate(U)

# Define states
ket_0 = Matrix([[1], [0]])
ket_1 = Matrix([[0], [1]])
ket_p = Matrix([[1], [1]]) / sqrt(2)
ket_m = Matrix([[1], [-1]]) / sqrt(2)
ket_i = Matrix([[1], [I]]) / sqrt(2)
ket_ni = Matrix([[1], [-I]]) / sqrt(2)

def swap(state, i1, i2):
    """Swap two qubits in a state vector."""
    n = log(state.shape[0], 2)
    assert n == int(n)
    n = int(n)
    op_qc = QuantumCircuit(n)
    op_qc.swap(n - 1 - i1, n - 1 - i2)
    op = Matrix(Operator(op_qc))
    return simplify(op * state)

state_og = Matrix([a, b, c, d])
state_tp = swap(TensorProduct(Matrix([1, 0, 0, 1]) / sqrt(2), state_og), 0, 2)
state_tk = swap(TensorProduct(1/sqrt(1+k**2) * (TensorProduct(ket_0, ket_0) + k * TensorProduct(ket_1, ket_1)), state_og), 0, 2)
state_pp = swap(TensorProduct(ket_p, ket_p, state_og), 0, 2)
state_nn = swap(TensorProduct(ket_m, ket_m, state_og), 0, 2)
state_ii = swap(TensorProduct(ket_i, ket_i, state_og), 0, 2)
state_ni = swap(TensorProduct(ket_ni, ket_ni, state_og), 0, 2)

# Define density matricies
s_og = state_og * state_og.H
s_tp = state_tp * state_tp.H
s_tk = state_tk * state_tk.H
s_pp = state_pp * state_pp.H
s_nn = state_nn * state_nn.H
s_ii = state_ii * state_ii.H
s_ni = state_ni * state_ni.H

def _trace_out(state, idx):
    """Trace out a qubit from a state vector.
    
    Args:
        state: The state vector to trace out.
        idx: The index of the qubit to trace out."""
    n_qubits = int(log(state.shape[0], 2))
    tp1 = [eye(2)] * idx + [ket_0.H] + [eye(2)] * (n_qubits - idx - 1)
    tp2 = [eye(2)] * idx + [ket_0] + [eye(2)] * (n_qubits - idx - 1)
    tp3 = [eye(2)] * idx + [ket_1.H] + [eye(2)] * (n_qubits - idx - 1)
    tp4 = [eye(2)] * idx + [ket_1] + [eye(2)] * (n_qubits - idx - 1)
    return simplify(TensorProduct(*tp1) * state * TensorProduct(*tp2) +
                    TensorProduct(*tp3) * state * TensorProduct(*tp4))

def trace_out(state, idx=(0,)):
    """Trace out qubits from a state vector.

    Args:
        state: The state vector to trace out.
        idx: The indices of the qubits to trace out."""
    for i in sorted(idx, reverse=True):
        state = _trace_out(state, i)
    return state

def subs_u(target, U_gate):
    """Substitute a U gate into a target expression."""
    return simplify(target.subs({u0: U_gate[0, 0], u1: U_gate[0, 1], u2: U_gate[1, 0], u3: U_gate[1, 1]}))

### Gate Cutting

Decomposition of a controlled-{X, Y, Z, H} gate based on the CZ decomposition from "[Constructing a virtual two-qubit gate by sampling single-qubit operations](https://doi.org/10.1088/1367-2630/abd7bc)"

![](figures/cu-decomposition.png)

In [2]:
def decomposition_density_matries(A1, A2, state=s_og):
    """
        This function returns the density matrices of the 10 circuits depicted above applied to an arbitrary 2-qubit state.
        The decomposition is from https://doi.org/10.1088/1367-2630/abd7bc.

        Args:
            A1: The first gate matrix (Replaces the Z in the above image).
            A2: The second gate matrix (Replaces the U gate in the above image).
    """
    t1 = TensorProduct(Matrix(exp(I * pi * A1 / 4)), Matrix(exp(I * pi * A2 / 4)))
    s1 = t1 * state * t1.H
    t2 = TensorProduct(Matrix(exp(-I * pi * A1 / 4)), Matrix(exp(-I * pi * A2 / 4)))
    s2 = t2 * state * t2.H
    a1, a2 = symbols("alpha_1 alpha_2")
    t3 = TensorProduct((eye(2) + a1 * A1) / 2, Matrix(exp(I * (a2 + 1) * pi * A2 / 4)))
    t4 = TensorProduct(Matrix(exp(I * (a1 + 1) * pi * A1 / 4)), (eye(2) + a2 * A2) / 2)
    s3 = t3.subs({a1: -1, a2: -1}) * state * t3.subs({a1: -1, a2: -1}).H
    s4 = t4.subs({a1: -1, a2: -1}) * state * t4.subs({a1: -1, a2: -1}).H
    s5 = t3.subs({a1: -1, a2: 1}) * state * t3.subs({a1: -1, a2: 1}).H
    s6 = t4.subs({a1: -1, a2: 1}) * state * t4.subs({a1: -1, a2: 1}).H
    s7 = t3.subs({a1: 1, a2: -1}) * state * t3.subs({a1: 1, a2: -1}).H
    s8 = t4.subs({a1: 1, a2: -1}) * state * t4.subs({a1: 1, a2: -1}).H
    s9 = t3.subs({a1: 1, a2: 1}) * state * t3.subs({a1: 1, a2: 1}).H
    s10 = t4.subs({a1: 1, a2: 1}) * state * t4.subs({a1: 1, a2: 1}).H

    return s1, s2, s3, s4, s5, s6, s7, s8, s9, s10

# Test the decomposition for gates X, Y, Z, and H
for G in (X, Y, Z, H):
    d0, d1, d2, d3, d4, d5, d6, d7, d8, d9 = decomposition_density_matries(Z, G)
    decomposition = (d0 + d1 - d2 - d3 + d4 + d5 + d6 + d7 - d8 - d9) / 2

    CG = controlled_gate(G)
    result = simplify(CG * s_og * CG.H)
    decomposed_result = subs_u(decomposition, G)
    assert result.equals(decomposed_result)

### Gate Teleportation with NME States

Gate teleportation circuit to teleport a controlled-U gate from "[Optimal local implementation of nonlocal quantum gates](https://doi.org/10.1103/PhysRevA.62.052317)". In the image below the measurements are deferred and the resource state is an arbitrary 2-qubit NME state $\ket{\phi_{1,2}}$.

![](figures/gate-teleportation.png)

\begin{align}
    \rho_{\ket{++}} & = \rho_{\ket{--}} \\
    \rho_{\ket{ii}} & = \rho_{\ket{-i-i}} \\
\end{align}

In [3]:
def gate_tp(state, U_gate=CU):
    """
        This function returns the density matrix of the gate teleportation circuit with specified resource state applied to an arbitrary state.

        Args:
            state: The initial state for all 4 qubits.
            U_gate: The matrix of the controlled gate to teleport.
    """
    # CNOT(0, 1)
    op = TensorProduct(CX, eye(4))
    t1 = op * state
    # CNOT(1, 2)
    op = TensorProduct(eye(2), CX, eye(2))
    t2 = op * t1
    # U(2, 3)
    op = TensorProduct(eye(4), U_gate)
    t3 = op * t2
    # H(2)
    op = TensorProduct(eye(4), H, eye(2))
    t4 = op * t3
    # CZ(2, 0)
    op_qc = QuantumCircuit(4)
    op_qc.cz(1, 3)
    op = Matrix(Operator(op_qc))
    t5 = op * t4
    # trace out qubits 1, 2 (measure & classical communication)
    t5 = trace_out(t5 * t5.H, (1, 2))
    return t5

result_og = CU * s_og * CU.H
result_tp = gate_tp(state_tp)
result_tk = gate_tp(state_tk)
result_pp = gate_tp(state_pp)
result_nn = gate_tp(state_nn)
result_ii = gate_tp(state_ii)
result_ni = gate_tp(state_ni)

# Check that the original gate teleportation works (resource state (|00> + |11>)/sqrt(2))
assert result_og.equals(result_tp)

# Equation (1): Check equivilance for resource states |++> and |-->)
assert result_pp.equals(result_nn)
# Equation (2): Check equivilance for resource states |ii> and |-i-i>
assert result_ii.equals(result_ni)

### Gate Cutting with NME Gate Teleportation

![](figures/gate-cut.png)

\begin{equation*}
    E_{nme} := \rho_{\ket{\Phi^+}} - \rho_{\ket{\Phi^k}} = \underbrace{\frac{(k-1)^2}{k^2+1}}_{c} \underbrace{\left[\begin{matrix}0 & \rho_{0,1}\\\rho_{1,0} & 0\end{matrix}\right]}_{E_{\rho}}
\end{equation*}

\begin{equation}
    \rho_{\ket{\Phi^+}} = \rho_{\ket{\Phi^k}} + \underbrace{c \left(\rho_{\ket{++}} - \rho_{\ket{ii}}\right)}_{E_{nme}}
\end{equation}

In [4]:
# Define multiplication factor for compensation circuits
c = (k-1)**2/(k**2+1)
# Define error
error = simplify(result_pp - result_ii)
weighted_error = c * error
display(Math('E_{\\rho} = ' + latex(error)))
# Density matrix of gate teleportation with NME state
nme_tp = simplify(result_tk + weighted_error)
# Check Equation (1)
assert result_tp.equals(nme_tp)

<IPython.core.display.Math object>

### Improving the Compensation Circuits

![](figures/ancilla-elimination.png)

\begin{equation}
    g^{4} - e^{4 i \varphi} \overline{g}^{4} \overset{!}{=} 0
\end{equation}

---

\begin{align}
    G=I & \iff g=1,\varphi=2\pi n \\
    G=Z & \iff g=1,\varphi=2\pi n + \pi \\
    G=S & \iff g=1,\varphi=2\pi n + \frac{\pi}{2} \\
    G=S^\dagger & \iff g=1,\varphi=2\pi n - \frac{\pi}{2}
\end{align}

---

\begin{align}
    \rho_{\ket{++}} & = \rho_I' \\
    \rho_{\ket{--}} & = \rho_Z' \\
    \rho_{\ket{ii}} & = \rho_S' \\
    \rho_{\ket{-i-i}} & = \rho_{S^\dagger}'
\end{align}

In [5]:
g = symbols('g', complex=True)
phi = symbols('phi', real=True)
G = Matrix([[g, 0], [0, exp(I * phi)*conjugate(g)]])

def comp_circ(state, G_gate=G):
    """
        This function returns the density matrix of the compensation circuit applied to the state [a, b, c, d].

        Args:
            state: The initial state for all 3 qubits.
            G_gate: The matrix of the gate to transform Bob's ancilla qubit.
    """
    # G(1)
    op = TensorProduct(eye(2), G_gate, eye(2))
    t1 = op * state
    # CU(1, 2)
    op = TensorProduct(eye(2), CU)
    t2 = op * t1
    # H(1)
    op = TensorProduct(eye(2), H, eye(2))
    t3 = op * t2
    # CZ(1, 0)
    op = TensorProduct(CZ, eye(2))
    t4 = op * t3
    # G(0)
    op = TensorProduct(G_gate, eye(4))
    t5 = op * t4
    # trace out qubit 1
    t5 = trace_out(t5 * t5.H, (1,))
    return t5

# run the gate teleportation circuit with resource state (G\otimes G) |++>
state_tp_gp = TensorProduct(eye(2), G, G, eye(2)) * state_pp
result_tp_gp = gate_tp(state_tp_gp)

# run the compensation circuit with ancilla state G |+>
state_comp_gp = swap(TensorProduct(ket_p, state_og), 0, 1)
result_comp_gp = comp_circ(state_comp_gp)

factor = (g**4 - exp(4*I*phi) * conjugate(g)**4) * exp(-2*I*phi) / 4
fac_error = simplify((result_comp_gp - result_tp_gp) / factor)
display(Math('\\rho_G - \\rho_{(G\\otimes G)\ket{++}} = ' + latex(factor) + latex(fac_error)))

# Check that the improvement works for resource states |++>, |-->, |ii>, and |-i-i>
for s in ({g: 1, phi: 0}, {g: 1, phi: pi}, {g: 1, phi: pi/2}, {g: 1, phi: -pi/2}):
    assert result_tp_gp.subs(s).equals(result_comp_gp.subs(s))

<IPython.core.display.Math object>

### Decomposing the Compensation Circuits

![](figures/error-decomposition.png)

In [6]:
def decomp1(s):
    """
        This function returns the density matrix of the decomposed compensation circuit C(|+>) applied to an arbitrary state.

        Args:
            s: The initial state.
    """
    op1 = TensorProduct(eye(2), (eye(2)+U)/2)
    s1 = op1 * s * op1.H
    op2 = TensorProduct(Z, (eye(2)-U)/2)
    s2 = op2 * s * op2.H
    return simplify(s1 + s2)

def decomp2(s):
    """
        This function returns the density matrix of the decomposed compensation circuit C(|-i>) applied to an arbitrary state.

        Args:
            s: The initial state.
    """
    op1 = TensorProduct(SGATE.H, (eye(2)-I*U)/sqrt(2))
    s1 = op1 * s * op1.H
    op2 = TensorProduct(SGATE, (eye(2)+I*U)/sqrt(2))
    s2 = op2 * s * op2.H
    return simplify(s1 + s2) / 2

# Check that the decomposition works for arbitrary CU gate
decomposition1 = decomp1(s_og)
decomposition2 = decomp2(s_og)
assert decomposition1.equals(result_pp)
assert decomposition2.equals(result_ii)
assert error.equals(decomposition1 - decomposition2)

![](figures/cu-error-decomposition.png)

In [7]:
# Check decompositions from image above for gates X, Y, Z, and H
for u in (X, Y, Z, H):
    s0, s1, s2, s3, s4, s5, s6, s7, s8, s9 = decomposition_density_matries(Z, u)
    decomposition_p = (s2 - s3 + s4 + s5 + s6 + s7 + s8 - s9) / 2
    decomposition_i = (-s0 - s1 + s2 + s3 + s4 + s5 + s6 + s7 + s8 + s9) / 2
    decomposition_e = ((s0 + s1) / 2 - s3 - s9)

    assert subs_u(decomposition_p, u).equals(subs_u(result_pp, u))
    assert subs_u(decomposition_i, u).equals(subs_u(result_ii, u))
    assert subs_u(decomposition_e, u).equals(subs_u(error, u))